In [29]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import gym_ignition_models

In [30]:
def SX2DM(sx):
    return cs.DM(sx)

## Import the robot .urdf

In [31]:
urdf_path = gym_ignition_models.get_model_file("iCubGazeboV2_5")
print(urdf_path)
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
kinDyn = KinDynComputations(urdf_path, joints_name_list, root_link)

/home/carlotta/.local/lib/python3.8/site-packages/gym_ignition_models/iCubGazeboV2_5/icub.urdf


The class `KinDynComputations` contains some algorithms for computing kinematics and dynamics quantities.

In [32]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5

## Mass Matrix

In [33]:
M = kinDyn.mass_matrix_fun()
print('Mass matrix:\n', SX2DM(M(H_b, s)))

ixx: 0.0139679
ixy: -5.93488e-06
ixz: -1.55458e-05
iyy: 0.0202112
iyz: -0.00170968
izz: 0.0249124
5.09143
[0.0248821, 0.000103947, -0.0440806]
[1.57079632679, 0.0, -1.57079632679]
tutto bene
[[ 5.09143000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.24433289e-01 -5.29238874e-04]
 [ 0.00000000e+00  5.09143000e+00  0.00000000e+00  2.24433289e-01
   0.00000000e+00  1.26685470e-01]
 [ 0.00000000e+00  0.00000000e+00  5.09143000e+00  5.29238874e-04
  -1.26685470e-01  0.00000000e+00]
 [ 0.00000000e+00  2.24433289e-01  5.29238874e-04  3.48056091e-02
  -2.87143745e-05  7.29405155e-03]
 [-2.24433289e-01  0.00000000e+00 -1.26685470e-01 -2.87143745e-05
   2.70132546e-02  2.92640471e-05]
 [-5.29238874e-04  1.26685470e-01  0.00000000e+00  7.29405155e-03
   2.92640471e-05  2.33634556e-02]]
ixx: 0.000404268
ixy: -1.26852e-06
ixz: 1.11055e-06
iyy: 0.000574741
iyz: -7.69474e-07
izz: 0.000580025
0.919978
[-0.0005145, -0.0380753, -7.1e-05]
[1.57079632679, 0.0, 1.57079632679]
tutto bene
[[ 9.19

## Centroidal Momentum Matrix

In [34]:
Jcm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix:\n', SX2DM(Jcm(H_b, s)))

ixx: 0.0139679
ixy: -5.93488e-06
ixz: -1.55458e-05
iyy: 0.0202112
iyz: -0.00170968
izz: 0.0249124
5.09143
[0.0248821, 0.000103947, -0.0440806]
[1.57079632679, 0.0, -1.57079632679]
tutto bene
[[ 5.09143000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.24433289e-01 -5.29238874e-04]
 [ 0.00000000e+00  5.09143000e+00  0.00000000e+00  2.24433289e-01
   0.00000000e+00  1.26685470e-01]
 [ 0.00000000e+00  0.00000000e+00  5.09143000e+00  5.29238874e-04
  -1.26685470e-01  0.00000000e+00]
 [ 0.00000000e+00  2.24433289e-01  5.29238874e-04  3.48056091e-02
  -2.87143745e-05  7.29405155e-03]
 [-2.24433289e-01  0.00000000e+00 -1.26685470e-01 -2.87143745e-05
   2.70132546e-02  2.92640471e-05]
 [-5.29238874e-04  1.26685470e-01  0.00000000e+00  7.29405155e-03
   2.92640471e-05  2.33634556e-02]]
ixx: 0.000404268
ixy: -1.26852e-06
ixz: 1.11055e-06
iyy: 0.000574741
iyz: -7.69474e-07
izz: 0.000580025
0.919978
[-0.0005145, -0.0380753, -7.1e-05]
[1.57079632679, 0.0, 1.57079632679]
tutto bene
[[ 9.19

## Center of mass position

In [35]:
CoM = kinDyn.CoM_position_fun()
print('Center of Mass position:\n', SX2DM(CoM(H_b, s)))

Center of Mass position:
 [0.515611, 1.89538, -2.42293]


## Total Mass

In [36]:
print('Total mass:\n', kinDyn.get_total_mass())

Total mass:
 33.06167270000001


## Jacobian

In [37]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', SX2DM(J(H_b, s)))

Jacobian of the left sole:
 
[[1, 0, 0, 0, 0.233402, -0.104594, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.137593, 0.0406683, 0.182952, 0.0463936, -0.0169478, 0.0590439, 0, 0, 0, 0, 0, 0], 
 [0, 1, 0, -0.233402, 0, 0.0868809, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.100628, 0.0912684, 0.04696, -0.0094643, 0.0206949, -0.00813624, 0, 0, 0, 0, 0, 0], 
 [0, 0, 1, 0.104594, -0.0868809, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0926634, -0.0492641, 0.00922781, -0.245424, 0.0474163, 0.0246549, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.619346, -0.654279, -0.110572, -0.909489, 0.909489, 0.311307, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.781947, 0.563677, 0.588202, -0.384911, 0.384911, -0.380135, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0704931, 0.50417, -0.80112, -0.157081, 0.157081, -0.870968, 0, 0, 0, 0, 0, 0]]


## Relative jacobian

In [38]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', SX2DM(J_r(s)))

Jacobian between the root link and left sole:
 
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0381904, 0.016518, 0.058197, 0.213628, -0.0533472, 0.0124818, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.90714e-15, -0.100028, -0.14938, -0.0386338, -0.00234318, -0.0284685, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.190226, 0.0461781, -0.100301, 0.123877, -0.0106017, -0.0565155, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.07045e-13, -0.941575, 0.301124, -0.137467, 0.137467, 0.97991, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1.94727e-13, -0.447933, 0.853195, -0.853195, 0.0430407, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.336802, 0.841831, 0.503152, -0.503152, 0.194738, 0, 0, 0, 0, 0, 0]]


## Forward kinematics

In [39]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', SX2DM(H(H_b, s)))

Forward kinematics to the left sole:
 
[[0.311307, 0.91541, -0.255172, 0.585051], 
 [-0.380135, -0.126143, -0.916289, 2.0245], 
 [-0.870968, 0.382247, 0.30871, -2.18082], 
 [0, 0, 0, 1]]


## Bias force term

In [40]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot)))

ixx: 0.0139679
ixy: -5.93488e-06
ixz: -1.55458e-05
iyy: 0.0202112
iyz: -0.00170968
izz: 0.0249124
5.09143
[0.0248821, 0.000103947, -0.0440806]
[1.57079632679, 0.0, -1.57079632679]
tutto bene
[[ 5.09143000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.24433289e-01 -5.29238874e-04]
 [ 0.00000000e+00  5.09143000e+00  0.00000000e+00  2.24433289e-01
   0.00000000e+00  1.26685470e-01]
 [ 0.00000000e+00  0.00000000e+00  5.09143000e+00  5.29238874e-04
  -1.26685470e-01  0.00000000e+00]
 [ 0.00000000e+00  2.24433289e-01  5.29238874e-04  3.48056091e-02
  -2.87143745e-05  7.29405155e-03]
 [-2.24433289e-01  0.00000000e+00 -1.26685470e-01 -2.87143745e-05
   2.70132546e-02  2.92640471e-05]
 [-5.29238874e-04  1.26685470e-01  0.00000000e+00  7.29405155e-03
   2.92640471e-05  2.33634556e-02]]
ixx: 0.000404268
ixy: -1.26852e-06
ixz: 1.11055e-06
iyy: 0.000574741
iyz: -7.69474e-07
izz: 0.000580025
0.919978
[-0.0005145, -0.0380753, -7.1e-05]
[1.57079632679, 0.0, 1.57079632679]
tutto bene
[[ 9.19

## Coriolis term

In [41]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot)))

ixx: 0.0139679
ixy: -5.93488e-06
ixz: -1.55458e-05
iyy: 0.0202112
iyz: -0.00170968
izz: 0.0249124
5.09143
[0.0248821, 0.000103947, -0.0440806]
[1.57079632679, 0.0, -1.57079632679]
tutto bene
[[ 5.09143000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.24433289e-01 -5.29238874e-04]
 [ 0.00000000e+00  5.09143000e+00  0.00000000e+00  2.24433289e-01
   0.00000000e+00  1.26685470e-01]
 [ 0.00000000e+00  0.00000000e+00  5.09143000e+00  5.29238874e-04
  -1.26685470e-01  0.00000000e+00]
 [ 0.00000000e+00  2.24433289e-01  5.29238874e-04  3.48056091e-02
  -2.87143745e-05  7.29405155e-03]
 [-2.24433289e-01  0.00000000e+00 -1.26685470e-01 -2.87143745e-05
   2.70132546e-02  2.92640471e-05]
 [-5.29238874e-04  1.26685470e-01  0.00000000e+00  7.29405155e-03
   2.92640471e-05  2.33634556e-02]]
ixx: 0.000404268
ixy: -1.26852e-06
ixz: 1.11055e-06
iyy: 0.000574741
iyz: -7.69474e-07
izz: 0.000580025
0.919978
[-0.0005145, -0.0380753, -7.1e-05]
[1.57079632679, 0.0, 1.57079632679]
tutto bene
[[ 9.19

## Gravity term

In [42]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s)))

ixx: 0.0139679
ixy: -5.93488e-06
ixz: -1.55458e-05
iyy: 0.0202112
iyz: -0.00170968
izz: 0.0249124
5.09143
[0.0248821, 0.000103947, -0.0440806]
[1.57079632679, 0.0, -1.57079632679]
tutto bene
[[ 5.09143000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.24433289e-01 -5.29238874e-04]
 [ 0.00000000e+00  5.09143000e+00  0.00000000e+00  2.24433289e-01
   0.00000000e+00  1.26685470e-01]
 [ 0.00000000e+00  0.00000000e+00  5.09143000e+00  5.29238874e-04
  -1.26685470e-01  0.00000000e+00]
 [ 0.00000000e+00  2.24433289e-01  5.29238874e-04  3.48056091e-02
  -2.87143745e-05  7.29405155e-03]
 [-2.24433289e-01  0.00000000e+00 -1.26685470e-01 -2.87143745e-05
   2.70132546e-02  2.92640471e-05]
 [-5.29238874e-04  1.26685470e-01  0.00000000e+00  7.29405155e-03
   2.92640471e-05  2.33634556e-02]]
ixx: 0.000404268
ixy: -1.26852e-06
ixz: 1.11055e-06
iyy: 0.000574741
iyz: -7.69474e-07
izz: 0.000580025
0.919978
[-0.0005145, -0.0380753, -7.1e-05]
[1.57079632679, 0.0, 1.57079632679]
tutto bene
[[ 9.19